In [35]:
import numpy as np
from dataclasses import dataclass


In [36]:
dt = 0.05
FF = np.array([[1, 0, dt, 0], [0, 1, 0, dt], [0, 0, 1, 0], [0, 0, 0, 1]], dtype=np.float64)

Pnn0 = np.identity(4, dtype=np.float64)

qa = np.sqrt(10)
Qa = np.array([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]], dtype=np.float64) * qa
QQ = FF @ Qa @ FF.T

HH = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], dtype=np.float64)

vn = np.array([np.sqrt(10), np.sqrt(10)], dtype=np.float64)[:, np.newaxis]
Rn = vn @ vn.T

@dataclass
class xylist:
    sources: np.ndarray
    
    def __post_init__(self):
        
        self.sortedSources = self.sources[self.sources[:,2].argsort()[::-1]] # sort by flux
    
    def topNsources(self, numSources):
        return self.sortedSources[0:numSources, :]

In [37]:
with open("xyIms.dat", 'rb') as ff:
    xyIms = np.load(ff, allow_pickle=True)

In [122]:
# print(xyIms[0].sources[0:10, 0:2])
def source2colvec(sourceArray):

    nS = len(sourceArray.flat)
    stateVec = np.zeros((nS), dtype=np.float64)[:, np.newaxis]
    for idx, xx in enumerate(sourceArray.flat):
        stateVec[idx, 0] = xx
    
    return stateVec

def makeFFmatrix(sourceArray, velArray, dt):
    
    numSources = sourceArray.shape[0]
    numVels = sourceArray.shape[0]
    
    fShape = numSources * 2 + 2
    FF = np.identity(fShape, dtype=np.float64)
    
    dtIdx = (0, 2)
    for idx in range(numSources*2):
        FF[dtIdx[0], dtIdx[1]] = dt
        dtIdx = [vv + 1 for vv in dtIdx]
        # print(FF[0, 2])
    
    return FF

def makeHHmatrix(numSources):
    
    HH = np.identity(numSources*2, dtype=np.float64)
    zc = np.zeros((numSources*2, 2), dtype=np.float64)
    
    return np.concatenate((HH, zc), axis=1)

def makeQmat(FF, qa):
    
    # Qa = np.array([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]], dtype=np.float64) * qa
    Qa = np.zeros(FF.shape, dtype=np.float64)
    Qa[FF.shape[0]-1, FF.shape[1]-1] = qa
    Qa[FF.shape[0]-2, FF.shape[1]-2] = qa
    
    # np.set_printoptions(precision=3)
    # print(Qa)
    QQ = FF @ Qa @ FF.T
    return QQ

#====================================#
# init things
#====================================#

nsources = 4
sourceArray = xyIms[0].sources[0:nsources, 0:2]
sv = source2colvec(sourceArray)
vv = np.array([-10, -10], dtype=np.float64)[:, np.newaxis]

stateVec0 = np.concatenate((sv, vv), 0)
PPnn0 = np.identity(stateVec0.shape[0], dtype=np.float64)

## start loop over sources
xxTimeHist = []
ppTimeHist = []

Xnn = stateVec0
Pnn = PPnn0

HH = makeHHmatrix(nsources)
vn = np.array([np.sqrt(10)]*nsources*2, dtype=np.float64)[:, np.newaxis]
Rn = vn @ vn.T

for idx, xy in enumerate(xyIms[1:]):
    
    # sourceArray = xy.sources[0:nsources, 0:2]
    sourceArray = xyIms[idx-1].sources[0:nsources, 0:2]
    
    FF = makeFFmatrix(sourceArray, Xnn[-2:], dt)
    QQ = makeQmat(FF, qa)
    
    Pn1 = FF @ Pnn @ FF.T + QQ
    Xn1 = FF @ Xnn
    
    
    KK = Pn1 @ HH.T @ np.linalg.inv(HH @ Pn1 @ HH.T + Rn)
    
    sourceArray = xy.sources[0:nsources, 0:2]
    zn = source2colvec(sourceArray)
    Xn = Xn1 + KK @ (zn - HH@Xn1)
    
    Pn = (np.identity(Xn.shape[0]) - KK @ HH) @ Pn1 @ (np.identity(Xn.shape[0]) - KK @ HH) + KK @ Rn @ KK.T
    
    break
                                                       

# print(stateVec)
# print(makeFFmatrix(sourceArray, vv, dt))
# print(makeHHmatrix(nsources))



In [123]:
Pn

array([[ 0.138,  0.138,  0.131,  0.131,  0.132,  0.132,  0.132,  0.132,
         0.027,  0.027],
       [ 0.138,  0.138,  0.131,  0.131,  0.132,  0.132,  0.132,  0.132,
         0.027,  0.027],
       [ 0.138,  0.138,  0.131,  0.131,  0.132,  0.132,  0.132,  0.132,
         0.027,  0.027],
       [ 0.138,  0.138,  0.131,  0.131,  0.132,  0.132,  0.132,  0.132,
         0.027,  0.027],
       [ 0.138,  0.138,  0.131,  0.131,  0.132,  0.132,  0.132,  0.132,
         0.027,  0.027],
       [ 0.138,  0.138,  0.131,  0.131,  0.132,  0.132,  0.132,  0.132,
         0.027,  0.027],
       [ 0.138,  0.138,  0.131,  0.131,  0.132,  0.132,  0.132,  0.132,
         0.027,  0.027],
       [ 0.138,  0.138,  0.131,  0.131,  0.132,  0.132,  0.132,  0.132,
         0.027,  0.027],
       [ 0.13 ,  0.13 ,  0.121,  0.123,  0.166,  0.123, -0.723,  0.128,
         4.125,  0.005],
       [ 0.13 ,  0.13 ,  0.123,  0.121,  0.123,  0.166,  0.128, -0.723,
         0.005,  4.125]])